In [ ]:
# @title Install Requirment
!pip install git+https://github.com/epg900/whisper.git
!pip install -U deep-translator
!pip install rich

from IPython.display import clear_output
clear_output()
import whisper
model = whisper.load_model("turbo")
clear_output()
#!whisper vid.mp4 --model medium --language it

In [ ]:
# @title Upload Video and Download Subtitle
destination_language = "fa" # @param ["en","fa","es","it","fr","du"]
#results = model.transcribe("vid.mp4", language="it", task="translate")
from deep_translator import GoogleTranslator
from rich.progress import track
from google.colab import files

filenameext = ""
%cd /content/
uploaded = files.upload()
if uploaded:
  filenameext = next(iter(uploaded))
else:
  linktxt = input("Enter Video Download Link: ")
  if linktxt != '':
    !wget {linktxt} -O video.mp4
  filenameext = "video.mp4"
filename = filenameext.split('.')[0]
clear_output()

result = model.transcribe(filenameext)

class dotdict(dict):
  def __getattr__(self,key):
     return self[key]
  def __setattr__(self,key,value):
    self[key] = value

def format_time(seconds_float):
    total_milliseconds = round(seconds_float * 1000)
    hours, remainder = divmod(abs(total_milliseconds), 3600000)
    minutes, remainder = divmod(remainder, 60000)
    seconds, milliseconds = divmod(remainder, 1000)

    sign = '-' if seconds_float < 0 else ''
    return f"{sign}{hours:02d}:{minutes:02d}:{seconds:02d},{milliseconds:03d}"

f = open(f"{filename}.srt",'w',encoding='utf-8')
f2 = open(f"{filename}_origlang.srt",'w',encoding='utf-8')
f3 = open(f"{filename}_origfa.srt",'w',encoding='utf-8')

for k in track(result['segments'], description="Processing..."):
  fatxt = GoogleTranslator(source='auto', target=destination_language).translate(k['text'])
  f.write(f"{k['id']}\n{format_time(k['start'])} --> {format_time(k['end'])}\n{fatxt}\n\n")
  f2.write(f"{k['id']}\n{format_time(k['start'])} --> {format_time(k['end'])}\n{k['text']}\n\n")
  f3.write(f"{k['id']}\n{format_time(k['start'])} --> {format_time(k['end'])}\n{k['text']}\n{fatxt}\n\n")

f.close()
f2.close()
f3.close()

files.download(f"{filename}.srt")
files.download(f"{filename}_origlang.srt")
files.download(f"{filename}_origfa.srt")

In [ ]:
# @title Youtube Viewer and Downloader
vidurl = "" # @param {type:"string"}
!pip install yt-dlp
!rm video.mp4
!yt-dlp -f 18 -o video.mp4 {vidurl}
!yt-dlp --get-description {vidurl} > video.txt
#!yt-dlp --write-comments --dump-single-json {vidurl} > video.txt
#!jq '.description'  video.txt
#!curl -F file=@"video.mp4" https://epfs2.eu.pythonanywhere.com/uploader > /dev/null
from IPython.display import clear_output
clear_output()
#print("https://epfs2.eu.pythonanywhere.com/video.mp4")
#import IPython
#display(IPython.display.Javascript("window.open('https://epfs2.eu.pythonanywhere.com/vid.mp4')"))